## Resource-Constrained Project Scheduling

This notebook demonstrates how to model and solve the classical Resource-Constrained Project Scheduling Problem 
using Constraint Programming with IBM’s CP Optimizer via the [`docplex.cp`](https://ibmdecisionoptimization.github.io/docplex-doc/cp/refman.html) Python API.


### Problem Definition

$$
\begin{aligned}
\min \quad & \max_{i \in [1..N]} \mathrm{endOf}(x_i) \qquad &\qquad & \text{(1)} \\
\text{s.t.} \quad
& \sum_{i \in [1..N]} \mathrm{pulse}(x_i, Q_{ik}) \le C_k, \qquad & \forall k \in [1..M] \quad & \text{(2)} \\
& \mathrm{endBeforeStart}(x_i, x_j), \qquad & \forall (i,j) \in P \quad & \text{(3)} \\
& \text{interval } x_i,\ \text{size} = PT_i, \qquad & \forall i \in [1..N] \quad & \text{(4)}
\end{aligned}
$$


We are given:

- A finite set of **tasks** indexed by $i \in [1..N]$
- A finite set of **renewable resources** indexed by $k \in [1..M]$

Each task $i$:
- requires a **known processing time** $PT_i > 0$,
- consumes $Q_{ik} \ge 0$ units of each resource $k$ while it runs.

Each resource $k$ has a **capacity** $C_k > 0$.

There is a set of **precedence relations** $P \subseteq [1..N]\times[1..N]$,  
where $(i,j)\in P$ means task $i$ must **finish before** task $j$ **starts**.

We decide **start times** for all tasks via interval variables $x_i$ (one per task, fixed size $PT_i$) (4).

Our goal is to build a feasible schedule such that:
- All precedence relations are respected (3).
- At any time, resource usage on each renewable resource does not exceed its capacity (2).
- The **makespan** $C_{\max}$ — the completion time of the last finishing task — is **minimized** (1).


#### Symbols and Notation

| Symbol / Function | Meaning | CPLEX CP Python Reference |
|---|---|---|
| $N$ | Number of tasks | — |
| $M$ | Number of renewable resources | — |
| $i$ | Task index ($i \in [1..N]$) | — |
| $k$ | Resource index ($k \in [1..M]$) | — |
| $PT_i$ | Processing time (duration) of task $i$ | — |
| $Q_{ik}$ | Units of resource $k$ required by task $i$ (while $i$ runs) | — |
| $C_k$ | Available capacity of resource $k$ | — |
| $x_i$ | Interval variable for task $i$ | — |
| $\mathrm{endOf}(x_i)$ | End time of task $i$ | [docplex.cp.modeler.end_of](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.end_of) |
| $\mathrm{pulse}(x_i, Q_{ik})$ | Time-varying usage of resource $k$ by $i$ while $x_i$ executes | [docplex.cp.modeler.pulse](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.pulse) |
| $\mathrm{endBeforeStart}(x_i, x_j)$ | Enforce $j$ to start after $i$ ends | [docplex.cp.modeler.end_before_start](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.end_before_start) |
| $\text{interval } x_i, \text{ size} = PT_i$ | Declare $x_i$ with fixed duration | [docplex.cp.expression.interval_var](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.expression.py.html#docplex.cp.expression.interval_var) |
| $\max_{i} \mathrm{endOf}(x_i)$ | Makespan of the project | [docplex.cp.modeler.end_of](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.end_of) |
| $\min C_{\max}$ | Objective: minimize makespan | [docplex.cp.modeler.minimize](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.minimize) |


### Implementation

This notebook is an adaptation of the *RCPSP* example from the  
[IBM Decision Optimization CPLEX Modeling for Python (DOcplex) Examples Repository](https://github.com/IBMDecisionOptimization/docplex-examples/tree/master).  

#### Imports

In [32]:
from docplex.cp.model import CpoModel, interval_var, end_of, pulse, end_before_start, minimize
import docplex.cp.utils_visu as visu

#### Reading the data file

In [19]:
filename = "../data/rcpsp/rcpsp_default.data"

# Available files are rcpsp_default, and different rcpsp_XXXXXX.
# First line contains the number of tasks, and the number of resources.
# Second line contains the capacities of the resources.
# The rest of the file consists of one line per task, organized as follows:
# - duration of the task
# - the demand on each resource (one integer per resource)
# - the number of successors followed by the list of successor numbers
with open(filename, "r") as f:
    N, M = [int(v) for v in f.readline().split()]  # N (tasks), M (resources)
    C = [int(v) for v in f.readline().split()] # capacities C_k, k=1..M
    raw_tasks = [[int(v) for v in f.readline().split()] for _ in range(N)]

# Durations of the tasks PT_i
PT = [raw_tasks[i][0] for i in range(N)]

# Demands on the resource Q_{ik}
Q = [raw_tasks[i][1:M+1] for i in range(N)]

# Successor lists (1-based in file) -> precedence arcs P = {(i, j)}
succ_lists = [raw_tasks[i][M+2:] for i in range(N)]
P = [(i, j - 1) for i in range(N) for j in succ_lists[i]]


In [22]:
print(f"Number of tasks (N): {N}")
print(f"Number of resources (M): {M}")
print(f"Resource capacities: {C}\n")

for i in range(N):
    print(f"Task {i}: duration={PT[i]}, demand={Q[i]}, successors={succ_lists[i]}")

print(f"\nPrecedence arcs P = {P}")

Number of tasks (N): 35
Number of resources (M): 4
Resource capacities: [5, 5, 5, 5]

Task 0: duration=0, demand=[0, 0, 0, 0], successors=[2, 3, 4]
Task 1: duration=5, demand=[1, 0, 0, 0], successors=[5, 6, 7]
Task 2: duration=9, demand=[1, 0, 0, 0], successors=[11, 12]
Task 3: duration=3, demand=[1, 2, 0, 0], successors=[8, 9, 10]
Task 4: duration=5, demand=[0, 4, 2, 0], successors=[11]
Task 5: duration=6, demand=[2, 1, 2, 0], successors=[11]
Task 6: duration=1, demand=[3, 2, 2, 0], successors=[16]
Task 7: duration=4, demand=[0, 0, 0, 3], successors=[13]
Task 8: duration=8, demand=[0, 1, 1, 0], successors=[13]
Task 9: duration=1, demand=[3, 2, 0, 0], successors=[13]
Task 10: duration=4, demand=[1, 2, 0, 0], successors=[14, 15, 16, 17]
Task 11: duration=2, demand=[0, 4, 2, 4], successors=[14, 15, 16, 17]
Task 12: duration=2, demand=[0, 1, 1, 0], successors=[19]
Task 13: duration=5, demand=[3, 3, 0, 0], successors=[18]
Task 14: duration=6, demand=[0, 2, 0, 1], successors=[18]
Task 15: d

#### Create model and variables

In [33]:
# Create a CP Optimizer model
mdl = CpoModel()

# (4) Define interval variables for each task x_i with duration PT_i
x = [interval_var(name=f"x_{i+1}", size=PT[i]) for i in range(N)]

#### Add constraints and define objective

In [ ]:
# (3) Precedences: endBeforeStart(x_i, x_j) for all (i, j) in P
mdl.add(end_before_start(x[i], x[j]) for (i, j) in P)




### Additional Resources